In [1]:
import numpy as np
from dataset.mnist import load_mnist

In [2]:
# 乗算レイヤー
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy
    
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
# 100 * 2 * 1.1 = 220
print(price)

# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
# dapple: リンゴの値段に対する支払額の微分
# dapple_num: リンゴの個数に関する支払額の微分
# dtax: 消費税に関する支払額の微分
print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


In [3]:
# 加算レイヤー
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy
    
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)
# (100 * 2 + 150 * 3) * 1.1 = 715
print(price)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
# dapple_num: リンゴの個数に関する支払額の微分
# dapple: リンゴの値段に対する支払額の微分
# dorange_num: オレンジの個数に関する支払額の微分
# dorange: オレンジの値段に対する支払額の微分
# dtax: 消費税に関する支払額の微分
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [4]:
# reluレイヤー
# forwardによってbackwardが生成される
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
    
layer = Relu()
arr = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(layer.forward(arr))
print(layer.backward( layer.forward(arr) ))

[[1. 0.]
 [0. 3.]]
[[1. 0.]
 [0. 3.]]


In [5]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx
    
layer = Sigmoid()
arr = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(layer.forward(arr))
print(layer.backward( layer.forward(arr) ))

[[0.73105858 0.37754067]
 [0.11920292 0.95257413]]
[[0.14373484 0.08872346]
 [0.01251554 0.04303412]]


In [6]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

# numpyは四則演算でブロードキャストするので注意
W = np.array([[1.0, -0.5], [-2.0, 3.0]])
b = np.array([1.0, -0.5])
layer = Affine(W, b)
x = np.array([1.0, -0.5])
print(layer.forward(x))
print(layer.backward( layer.forward(x) ))

[ 3.  -2.5]
[  4.25 -13.5 ]


In [7]:
# ソフトマックス関数は出力層
from common.functions import softmax, cross_entropy_error

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

layer = SoftmaxWithLoss()
x = np.array([1.0, -0.5])
t = np.array([1.0, -0.5])
print(layer.forward(x, t))
print(layer.backward( layer.forward(x, t) ))

0.20141315566974394
[-0.09121276  0.34121276]


In [8]:
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)
        # ソフトマックス関数で評価
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [9]:
# 数値微分とバックプロパゲーションで勾配の差はほぼないことを確認する
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size = 784, hidden_size=50, output_size=10)
x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

b1:2.9680358764228205e-09
W1:4.752709826149629e-10
b2:1.391927409985505e-07
W2:5.979151434037759e-09


In [10]:
# 実際に学習してみる
# バックプロパゲーションは数値微分とちがい解析的に勾配を解くため計算量が減って高速に動く
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size = 784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print( "===========" + "iteration:" + str(i) + "===========")
        print('train_acc: ' + str(train_acc))
        print('test_acc: ' + str(test_acc))

===========iteration:0===========
train_acc: 0.11953333333333334
test_acc: 0.1196
===========iteration:600===========
train_acc: 0.9
test_acc: 0.905
===========iteration:1200===========
train_acc: 0.9223166666666667
test_acc: 0.9222
===========iteration:1800===========
train_acc: 0.9339333333333333
test_acc: 0.9326
===========iteration:2400===========
train_acc: 0.9456666666666667
test_acc: 0.9422
===========iteration:3000===========
train_acc: 0.95235
test_acc: 0.949
===========iteration:3600===========
train_acc: 0.9570166666666666
test_acc: 0.9519
===========iteration:4200===========
train_acc: 0.9600666666666666
test_acc: 0.9562
===========iteration:4800===========
train_acc: 0.9639333333333333
test_acc: 0.9574
===========iteration:5400===========
train_acc: 0.9661
test_acc: 0.9598
===========iteration:6000===========
train_acc: 0.96765
test_acc: 0.9616
===========iteration:6600===========
train_acc: 0.9717666666666667
test_acc: 0.9638
===========iteration:7200===========
train_acc